# Создание функции по проверке наличия имени оборудования в письме

# 1. Постановка задачи

Существуют такие случаи, когда пользователь отправляет запрос в тех.поддержку, однако не упоминает наименования оборудования, с которым случилась проблема. Поэтому нам необходимо отправлять пользователю письмо с просьбой на корректировку своего ответа. С этой задачей могут справиться большие языковые модели.

# 2. Настройка окружения

Для того, чтобы решить данную задачу, импортируем необходимые библиотеки (open source) для работы, а также укажем, какую роль данные библиотеки играют в ходе исследования:
- **pandas**: используется для тестирования на реальных данных;
- **langchain**: используется для формирования промпта для модели;
- **langchain_ollama**: используется для доступа к локально равзернутым open source моделям.

In [6]:
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import ChatPromptTemplate
import pandas as pd

# 3. Выбор модели

Для выполнения задачи была выбрана LLM gemma2:9b, т.к. она была обучена на технических данных и способна хорошо понимать запросы пользователей, что является очень важным фактором для нашей задачи.

In [15]:
#base_url - путь к развернутой модели.

llm = OllamaLLM(
        model='gemma2',
        base_url='http://26.251.162.207:11434',
        temperature=0,
        format='json',
        num_ctx=8192,
        num_predict=128,
    )

# 4. Создание промпта

Внутри промпта мы обязательно задаем контекст того кем является LLM, в нашем случае это специалист пользовательской поддержки.

Затем мы сообщаем модели, что ей пришло письмо: оно состоит из заголовка(title) и текста письма(body).

Затем мы задаем модели задачу: оценивать, достаточно ли в письме информации, чтобы узнать название модели и приводим ему необходимые примеры названий, и условия, в которых они находятся.

Далее сообщаем формат ответа json с полями flag и answer:
 
flag: true означает, что можно определить название оборудования, а false - что нет. В случае flag: false модель также выдает непустой answer с просьбой об уточнении названия оборудования(в промпте имеются необходимые вежливые инструкции).

Далее выдаем инструкцию о том, чтобы модель отвечала кратко и обращала усиленное внимание именно на названия оборудования и задаем более точно формат ответа.

In [17]:
template = """Представь, что ты помощник специалиста пользовательской поддержки компании по производству и обслуживанию технического оборудования.

    Тебе поступило письмо по электронной почте от клиента.

    Заголовок письма: {title}.

    Текст письма: {body}.

    Проанализируй, достаточно ли информации из письма для того, чтобы узнать название модели оборудования. Пример названия модели оборудования: НК2-1404 или CX-1015-12.
    Оно также может быть окружено символами //.

    Ответ напиши в виде json-файла, в котором будет поле flag и поле answer.

    Если ты считаешь, что информации достаточно для того, чтобы понять название модели оборудования, тогда в поле flag напиши True, а поле answer сделай пустым.

    Если ты считаешь, что информации не достаточно для того, чтобы понять название модели оборудования, тогда в поле flag напиши False, а в поле answer напиши с точки зрения оператора поддержки, что необходимо добавить в письме, чтобы оно стало понятным.
    Отвечай так, как если бы ты лично общалась с клиентом. Будь вежливой, поздоровайся с ним, а потом укажи кратко по пунктам, чего не хватает. Не приводи примеры названий.

    Следуй инструкциям, отвечай кратко, но по делу. Думай дважды. Не сильно концетрируйся на контексте всего письма, только на название моделей. В ответе кроме json-файла ничего писать не нужно.

    Структура ответа:

    {{
    'flag': ...,
    'answer': ...,
    }}

    """

# 5. Чтение данных

С помощью библиотеки pandas прочитаем очищенные данные.

In [14]:
df = pd.read_csv('..\data\clear_data.csv')
df

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HONOR\AppData\Local\Temp\ipykernel_17156\3412834767.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('..\data\clear_data.csv')


,Тема,Описание,Тип оборудования,Точка отказа,Серийный номер
0,+BB0EPgRDBEIEMQRDBDo- +BCEEGAQbBBA- +BB0EGg-2-...,Здравствуйте.\r\n\r\n Сломался ...,Ноутбук,Блок питания,C222090381
1,/ шумит кулер / отклеились ножки,"Коллеги, добрый день.\r\n\r\nПрошу осуществить...",Ноутбук,Вентилятор,C223010310
2,10552. СИЛА HK-1404 // замена блоков питания //,"Добрый день! Не работают блоки питания, прошу ...",Ноутбук,Блок питания,"C223010423, C223011174"
3,10979. Ноутбук СИЛА НК2-1404 // не включается,Добрый день! Ноутбук не включается,Ноутбук,Материнская плата,C223012335
4,BIOS'ы,"Здравствуйте!\r\nПоделитесь, пожалуйста, после...",Ноутбук,Консультация,C223010345
...,...,...,...,...,...
180,по неисправной плашке памяти на dd203 - Enclos...,Добрый день!\r\nПросьба открыть заявку по неис...,СХД,Оперативная память,CKM00193800300
181,полосы на матрице НК2-3404,Полосы на матрице.,Ноутбук,Матрица,C223090893
182,полосы на экране и отклеились ножки,Переодически (было 3-4 раза) появляются полосы...,Ноутбук,Матрица,c223012503
183,ремонт ноутбука СИЛА НК2-1404,не работает порт usd и порт type-c на ноутбуке...,Ноутбук,Материнская плата,C223100081


# 6. Создание тестирующей функции

Создадим тестирующую функцию, которая принимает dataframe, index строки, шаблон промпта и LLM для выдачи результата работы модели.

In [ ]:
def check_prompt(df, index, prompt_template=template, llm=llm):
    title = df.iloc[index]['Тема']
    body = df.iloc[index]['Описание']


    prompt = ChatPromptTemplate.from_template(prompt_template)

    chain_ = prompt | llm 


    return chain_.invoke({
            'title': title,
            'body': body
        }), title, body

Пример работы модели. Как видно иногда модель может запросить еще доп данные, которые могут понадобиться оператору.

In [19]:
answer, title, body = check_prompt(df, 22, prompt_template=template, llm=llm)
print(answer)
print(title)
print(body)

{
    "flag": false,
    "answer": "Здравствуйте! Чтобы мы могли разобраться с ситуацией и помочь вам, пожалуйста, уточните модель вашего сервера.  Также будет полезно узнать версию прошивки." 
}   

Аварийная перезагрузка сервера,  UMA operation error
Аварийно перезагрузился сервер, в логе BMC появилось совпадающее по времени событие:
20	 Critical	2024-05-31 17:38:27 (UTC+3)
Intel ME has encountered UMA operation error. Details: UMA Read/Write timeout. Timeout occurred during copying data from/to UMA.

Просьба продиагностировать, возможно ли использовать данный сервер в продуктивном контуре
